In [1]:
#import libraries

import requests
from bs4 import BeautifulSoup
import glob
from datetime import datetime
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import calmap
%matplotlib inline

In [2]:
# Ministry of Health and Family Welfare url
URL = 'https://www.mohfw.gov.in'
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

html_thead = soup.find_all('thead')[-1]
html_th = [tr for tr in html_thead.find_all('tr')]

html_tbody = soup.find_all('tbody')[-1]
html_text = [tr for tr in html_tbody.find_all('tr')]

headings = []

for tr in html_th:
    th = tr.find_all(['th'])
    row = [i.text for i in th]
    headings.append(row)

print(headings)

content = []
for tr in html_text:
    td = tr.find_all(['td'])
    row = [i.text for i in td]
    content.append(row)
    
#print(content)

[['S. No.', 'Name of State / UT', 'Active Cases*', 'Cured/Discharged/Migrated*', 'Deaths**', 'Total Confirmed cases*']]


In [3]:
data= pd.DataFrame(content[:-6], columns=headings[0])

In [4]:
data

,S. No.,Name of State / UT,Active Cases*,Cured/Discharged/Migrated*,Deaths**,Total Confirmed cases*
0,1,Andaman and Nicobar Islands,10,40,0,50
1,2,Andhra Pradesh,5284,4599,119,10002
2,3,Arunachal Pradesh,126,22,0,148
3,4,Assam,2060,3762,9,5831
4,5,Bihar,1993,6104,56,8153
5,6,Chandigarh,90,322,6,418
6,7,Chhattisgarh,797,1553,12,2362
7,8,Dadra and Nagar Haveli and Daman and Diu,91,29,0,120
8,9,Delhi,24988,39313,2301,66602
9,10,Goa,703,205,1,909


In [5]:
# drop S.NO column
data = data.drop('S. No.', axis=1)

In [6]:
data.head()

,Name of State / UT,Active Cases*,Cured/Discharged/Migrated*,Deaths**,Total Confirmed cases*
0,Andaman and Nicobar Islands,10,40,0,50
1,Andhra Pradesh,5284,4599,119,10002
2,Arunachal Pradesh,126,22,0,148
3,Assam,2060,3762,9,5831
4,Bihar,1993,6104,56,8153


In [7]:
#rename column names
data = data.rename(columns={'Name of State / UT':'States'})
data = data.rename(columns=lambda x: re.sub('Total Confirmed cases \(Including .* foreign Nationals\) ',
                                                      'Total Cases',x))
data = data.rename(columns=lambda x: re.sub('Deaths \( more than 70% cases due to comorbidities \)',
                                                      'Deaths', x))
data = data.rename(columns={'Total Confirmed cases*': 'Total Cases'})
data = data.rename(columns={'Deaths**':'Death'})
data = data.rename(columns={'Cured/Discharged/Migrated*':'Recovered'})
data = data.rename(columns={'Deaths':'Death'})
data = data.rename(columns={'Active Cases*':'Active Cases'})

# remove extra characters from 'Name of State/UT' column
data['States'] = data['States'].str.replace('#', '')

# remove extra characters from 'Name of State/UT' column
data['Death'] = data['Death'].str.replace('#', '')

# drop active cases colum
data = data.drop('Active Cases', axis=1)

In [8]:
data.head()

,States,Recovered,Death,Total Cases
0,Andaman and Nicobar Islands,40,0,50
1,Andhra Pradesh,4599,119,10002
2,Arunachal Pradesh,22,0,148
3,Assam,3762,9,5831
4,Bihar,6104,56,8153


In [9]:
#add todays date and create new column
now  = datetime.now()
data['Date'] = now.strftime("%m/%d/%Y") 
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data.head()

,States,Recovered,Death,Total Cases,Date
0,Andaman and Nicobar Islands,40,0,50,2020-06-24
1,Andhra Pradesh,4599,119,10002,2020-06-24
2,Arunachal Pradesh,22,0,148,2020-06-24
3,Assam,3762,9,5831,2020-06-24
4,Bihar,6104,56,8153,2020-06-24


In [10]:
#add latitude and longitude
lat = {'Delhi':28.7041, 'Haryana':29.0588, 'Kerala':10.8505, 'Rajasthan':27.0238,
       'Telengana':18.1124, 'Uttar Pradesh':26.8467, 'Ladakh':34.2996, 'Tamil Nadu':11.1271,
       'Andhra Pradesh':15.9129, 'Odisha':20.9517, 'Uttarakhand':30.0668, 'West Bengal':22.9868, 
       'Puducherry': 11.9416, 'Chandigarh': 30.7333, 'Chhattisgarh':21.2787, 
       'Jammu and Kashmir':33.7782, 'Punjab':31.1471, 'Karnataka':15.3173, 
       'Maharashtra':19.7515, 'Gujarat': 22.2587, 
       'Himachal Pradesh': 31.1048, 'Madhya Pradesh': 22.9734, 'Bihar': 25.0961, 'Manipur':24.6637, 
       'Mizoram':23.1645, 'Goa': 15.2993, 'Andaman and Nicobar Islands': 11.7401, 'Assam' : 26.2006, 
       'Jharkhand': 23.6102, 'Arunachal Pradesh': 28.2180, 'Tripura': 23.9408, 'Nagaland': 26.1584, 
       'Meghalaya' : 25.4670}

long = {'Delhi':77.1025, 'Haryana':76.0856, 'Kerala':76.2711, 'Rajasthan':74.2179,
        'Telengana':79.0193, 'Uttar Pradesh':80.9462, 'Ladakh':78.2932, 'Tamil Nadu':78.6569,
        'Andhra Pradesh':79.7400, 'Odisha':85.0985, 'Uttarakhand':79.0193, 'West Bengal':87.8550, 
        'Puducherry': 79.8083, 'Chandigarh': 76.7794, 'Chhattisgarh':81.8661,
        'Jammu and Kashmir':76.5762, 'Punjab':75.3412, 'Karnataka':75.7139, 
        'Maharashtra':75.7139, 'Gujarat': 71.1924, 
        'Himachal Pradesh': 77.1734, 'Madhya Pradesh': 78.6569, 'Bihar': 85.3131, 'Manipur':93.9063, 
        'Mizoram':92.9376, 'Goa': 74.1240, 'Andaman and Nicobar Islands': 92.6586, 'Assam' : 92.9376, 
        'Jharkhand': 85.2799, 'Arunachal Pradesh': 94.7278, 'Tripura': 91.9882, 'Nagaland': 94.5624,
        'Meghalaya' : 91.3662}

data['Latitude'] = data['States'].map(lat)
data['Longitude'] = data['States'].map(long)

In [11]:
data.head()

,States,Recovered,Death,Total Cases,Date,Latitude,Longitude
0,Andaman and Nicobar Islands,40,0,50,2020-06-24,11.7401,92.6586
1,Andhra Pradesh,4599,119,10002,2020-06-24,15.9129,79.7400
2,Arunachal Pradesh,22,0,148,2020-06-24,28.2180,94.7278
3,Assam,3762,9,5831,2020-06-24,26.2006,92.9376
4,Bihar,6104,56,8153,2020-06-24,25.0961,85.3131


In [12]:
my_folder = './.daily_update/'
if not os.path.exists(my_folder):
    os.makedirs(my_folder)

file_name = my_folder + now.strftime("%Y_%m_%d") + '.csv'

In [13]:
data.to_csv(file_name, index=False)

In [14]:
csv_files = glob.glob(my_folder + '*.csv')

all_data = []
for i in csv_files:
    temp = pd.read_csv(i)
    all_data.append(temp)
    

In [15]:
final_data = pd.concat(all_data, ignore_index=True).sort_values(['Date'], ascending=True).reset_index(drop=True)
final_data['Date'] = pd.to_datetime(final_data['Date'])
#final_data.drop(final_data.index[[1005]])
final_data['Active'] = final_data['Total Cases'] - final_data['Recovered'] - final_data['Death']
final_data = final_data.sort_values(['Date', 'States']).reset_index(drop=True)
final_data

/Users/durgesh/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Date,Death,Latitude,Longitude,Recovered,States,Total Cases,Active
0,2020-04-21,0,11.7401,92.6586,11,Andaman and Nicobar Islands,16,5
1,2020-04-21,22,15.9129,79.7400,96,Andhra Pradesh,757,639
2,2020-04-21,0,28.2180,94.7278,1,Arunachal Pradesh,1,0
3,2020-04-21,1,26.2006,92.9376,19,Assam,35,15
4,2020-04-21,2,25.0961,85.3131,42,Bihar,114,70
5,2020-04-21,0,30.7333,76.7794,13,Chandigarh,26,13
6,2020-04-21,0,21.2787,81.8661,25,Chhattisgarh,36,11
7,2020-04-21,47,28.7041,77.1025,431,Delhi,2081,1603
8,2020-04-21,0,15.2993,74.1240,7,Goa,7,0
9,2020-04-21,77,22.2587,71.1924,131,Gujarat,2066,1858


In [16]:
final_data.to_csv('COVID19INDIA.csv', index=False)